## Описание проекта

Необходимо защитить данные клиентов страховой компании.
Задача - разработать метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. 

Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

### Инструкция по выполнению проекта

- загрузите и изучите данные.
- ответьте на вопрос и обоснуйте решение. Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
        a. Изменится. Приведите примеры матриц
        b. Не изменится.Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной

- предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.
- запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

### Описание данных

Набор данных находится в файле /datasets/insurance.csv

- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## 1. Загрузка данных

In [1]:
# Импортируем необходимые библитеки и методы

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Читаем исходный файл
data = pd.read_csv('/datasets/insurance.csv')

# Смотрим общую информацию
print('INFO')
display(data.info())
print('HEAD 10')
display(data.head(10))
print('DESCRIBE')
display(data.describe())
print('SPACES')
display(data.isnull().sum())
print('DUPLICATES')
display(data.duplicated().sum())
print('DIMENSION')
data.shape

INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

HEAD 10


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


DESCRIBE


...

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

DUPLICATES


153

DIMENSION


(5000, 5)

Наблюдаем незначительное, относительно всего объема выборки, количество дубликатов. Избавимся от них.

In [2]:
data = data.drop_duplicates().reset_index(drop = True)

Проверим результат.

In [3]:
print('DUPLICATES')
display(data.duplicated().sum())
print('DIMENSION')
data.shape

DUPLICATES


0

DIMENSION


(4847, 5)

Для повышения удобочитаемости изменим тип данных признаков "Возраст" и "Зарплата" с float64 на int64.

In [4]:
data['Возраст'] = data['Возраст'].astype('int')
data['Зарплата'] = data['Зарплата'].astype('int')

Проверим результат.

In [5]:
print('INFO')
display(data.info())

INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
Пол                  4847 non-null int64
Возраст              4847 non-null int64
Зарплата             4847 non-null int64
Члены семьи          4847 non-null int64
Страховые выплаты    4847 non-null int64
dtypes: int64(5)
memory usage: 189.5 KB


None

## 1. Загрузка данных. Выводы.

- исходный датасет представлен 5 000 записями и 5 признаками
- названия столбцов информативны и удобочитаемы
- пропущенных значений нет
- обнаружены 153 дубликата. В связи с их небольшим количеством, относительно общего объема выборки, принято решение об их удалении. Размерность конечного датасета составляет 4 847 записей и 5 признаков.
- для удобства изменили тип данных признаков "Возраст" и "Зарплата" с float64 на int64

## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:**

В случае умножения признаков на обратимую матрицу,  качество линейной регрессии не изменится.

**Обоснование:**

Пусть  $P$ — матрица, на которую умножаются признаки  
$P^{-1}$ — обратная ей матрица  
Известно что $P P^{-1} = E $ , где E - единичная матрица.  
Кроме того, известно что X E = E X = X.  
Тогда верна формула:  
$$
a = X P P^{-1} (X^T X)^{-1} X^T y
$$
или
$$
a = X E (X^T X)^{-1} X^T y
$$
или
$$
a = X (X^T X)^{-1} X^T y
$$
или
$$
a = X w
$$

Параметры линейной регрессии в исходной задаче и в преобразованной связаны через две основных формулы:  
- X E = E X = X  
- $P P^{-1} = E$  

Таким образом квадратная матрица P, размерностью совпадающая с количеством признаков в тренинговом датасете, может быть способом шифрования признаков, а обратная матрица $P^{-1}$ станет ключом к дешифровке данных

## 3. Алгоритм преобразования

**Алгоритм**

- генерируем случайную квадратную матрицу целых чисел $P$  (размерность равна количеству признаков в трейнинговом датасете)
- "шифруем" признаки в трейнинговом датасете умножением матриц $X P$  
- находим обратную ей матрицу $P^{-1}$ (ключ для дешифрования зашифрованных признаков в трейнинговом датасете)
- предсказывем целевой признак через алгоритм линейной регрессии двумя способами:

1) без шифрования признаков в трейнинговом датасете

2) с шифрованием, используя в качестве трейнингового датасета матрицу $X P P^{-1}$ 

- сравниваем качество моделирования в обоих сценариях, используя метрику R2

**Обоснование**

Пусть  $P$ — матрица, на которую умножаются признаки  
$P^{-1}$ — обратная ей матрица  

Известно что $P P^{-1} = E $ , где E - единичная матрица.  
Кроме того, известно что X E = E X = X.


Тогда верна формула:  
$$
a = X P P^{-1} (X^T X)^{-1} X^T y
$$
или
$$
a = X E (X^T X)^{-1} X^T y
$$
или
$$
a = X (X^T X)^{-1} X^T y
$$
или
$$
a = X w
$$

## 4. Проверка алгоритма

#### 4.1. Расчет качества линейной регрессии (sklearn) до преобразования матрицы признаков.  Метрика R2.

In [6]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

model = LinearRegression()
model.fit(features, target)
predicted = model.predict(features)
print('R2 =', r2_score(target, predicted))

R2 = 0.4302010046633359


#### 4.2. Преобразование матрицы признаков c помощью обратимой матрицы

Сгенерируем случайную квадратную матрицу P целых чисел размерностью равной количеству признаков в трейнинговом датасете и проверим работу формулы   
$$
E = P^{-1} P
$$

In [7]:
crypto= np.random.random_integers(1, 10, size=(data.drop('Страховые выплаты', axis=1).shape[1], data.drop('Страховые выплаты', axis=1).shape[1]))
print('Матрица случайных целых чисел P размерности = числу признаков \n', crypto, '\n' )
crypto_inverse = np.linalg.inv(crypto)
print('Обратная матрица  P**(-1) \n', crypto_inverse, '\n' )
print('Произведение обратимых матриц = E \n', (crypto_inverse @ crypto).round(0).astype('int'), '\n')

Матрица случайных целых чисел P размерности = числу признаков 
 [[ 6  8  5  8]
 [ 2  6 10  3]
 [10  5  5  2]
 [10  1  3  3]] 

Обратная матрица  P**(-1) 
 [[-0.02119816 -0.04009217  0.07235023  0.0483871 ]
 [ 0.06543779 -0.0718894   0.23317972 -0.25806452]
 [-0.07096774  0.1483871  -0.08387097  0.09677419]
 [ 0.11981567  0.00921659 -0.23502304  0.16129032]] 

Произведение обратимых матриц = E 
 [[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]] 



Работает корректно. Проработаем исходный датасет.

In [8]:
# Смотрим исходное состояние датасета
print('-- DATA --')
display(data.head())
print('\n')

# Шифруем
features_crypto = features @ crypto
features_crypto.columns = features.columns 
target_crypto = data['Страховые выплаты']

features_crypto_inverse = (features_crypto @ crypto_inverse).round(0).astype('int')
features_crypto_inverse.columns = features.columns
data_crypto = features_crypto
data_crypto['Страховые выплаты'] = target_crypto

print('-- DATA_CRYPTO --')
display(data_crypto.head())
print('\n')

# Дешифруем
data_decrypto = features_crypto_inverse
data_decrypto['Страховые выплаты'] = target_crypto

print('-- DATA_DECRYPTO --')
display(data_decrypto.head())

-- DATA --


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0




-- DATA_CRYPTO --


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,496098,248255,248418,99334,0
1,380102,190277,190463,76141,1
2,210058,105174,105290,42087,0
3,417062,208628,208716,83469,0
4,261062,130676,130785,52292,0




-- DATA_DECRYPTO --


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


#### 4.3. Расчет качества линейной регрессии (sklearn) после преобразования матрицы признаков. Метрика R2.

Проверим, что качество линейной регрессии остается неизменным после "шифрования" данных преобразованием матрицы признаков. В качестве трейнинговой базы используем произведение матриц:  
$$
X  P P^{-1}
$$

In [9]:
features_decrypto = features @ crypto @ crypto_inverse
features_decrypto.columns= features.columns

print('-- FEATURES_DECRYPTO --')
display(features_decrypto.round(0).astype('int').head())

model_2 = LinearRegression()
model_2.fit(features_decrypto, target_crypto)
predicted_crypto = model_2.predict(features_decrypto)

print('\n')
print('-- Метрика R2 --')

print('\n')
print('R2 =', r2_score(target, predicted))
print('R2 crypto =', r2_score(target_crypto, predicted_crypto))
print('\n')

print('-- Метрика MSE --')

def mse(target, predicted):
    return((target - predicted) ** 2).mean()

print('\n')
print('MSE =', mse(target, predicted))
print('MSE crypto =', mse(target_crypto, predicted_crypto))

-- FEATURES_DECRYPTO --


,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0




-- Метрика R2 --


R2 = 0.4302010046633359
R2 crypto = 0.4302010046633412


-- Метрика MSE --


MSE = 0.1252726382276536
MSE crypto = 0.12527263822765244


Основные метрики качества модели до и после преобразования матрицы признаков предельно близки.

## 5. Общий вывод

Практическим методом доказано, что метод умножения признаков на обратимую матрицу не влияет на качество предсказаний при использовании алгоритма линейной регрессии.

===================================================================================================================

#### Благодарю за внимание.